In [1]:
import pandas as pd
import numpy as np
import os
import json
import random
import re
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import  Input, LSTM, Embedding, Dense, Masking, Layer, Attention,Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

%matplotlib inline
warnings.filterwarnings('ignore') 

In [2]:
# JSON 파일이 있는 디렉토리 경로
directory_path = r"C:\Users\user\Downloads\TL_QA_seoul"

# JSON 데이터를 저장할 리스트
json_data_list = []

# 디렉토리 내의 모든 파일을 순회
for file_name in os.listdir(directory_path):
    if file_name.endswith(".json"):
        file_path = os.path.join(directory_path, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
                # 이미지 정보가 저장된 부분 삭제
                if 'images' in data:
                    del data['images']
                
                # 수정된 데이터를 리스트에 추가
                json_data_list.append(data)
                
                # 수정된 JSON 파일 저장
                with open(file_path, 'w', encoding='utf-8') as file:
                    json.dump(data, file, ensure_ascii=False, indent=4)
                
                # print(f"이미지 정보가 {file_name}에서 삭제되었습니다.")

        except FileNotFoundError:
            print(f"File {file_name} not found.")
        except json.JSONDecodeError:
            print(f"Error decoding JSON from file {file_name}.")

# 데이터 전처리

In [3]:
questions = []
answers = []

for item in json_data_list:
    # 'annotations' 리스트가 있는지 확인
    if 'annotations' in item:
        for annotation in item['annotations']:
            # 'question' 리스트가 있는지 확인
            if 'question' in annotation:
                for question in annotation['question']:
                    questions.append(question['question'])
                    answers.append(question['answer'])

print("Questions:", questions[:5])
print("Answers:", answers[:5])

Questions: ['이 곳의 주소를 알 수 있나요?', '이 곳의 주요 메뉴는 무엇이 있나요?', '이 곳에 인접한 시설이 있나요?', '주차시설이 있나요?', '이 곳의 주소는 어떻게 되나요?']
Answers: ['서울 강서구 우장산로16길 26', '생대구탕, 특대구탕', '강서구청, 커피바알아, 우리은행', '없음', '서울 강서구 우장산로16길 26']


In [4]:
# questions와 answers 리스트를 데이터프레임으로 변환
my_data = pd.DataFrame({
    'Source': questions,
    'Target': answers
})

# 데이터프레임 출력
print(my_data)

                       Source             Target
0           이 곳의 주소를 알 수 있나요?  서울 강서구 우장산로16길 26
1        이 곳의 주요 메뉴는 무엇이 있나요?         생대구탕, 특대구탕
2           이 곳에 인접한 시설이 있나요?  강서구청, 커피바알아, 우리은행
3                  주차시설이 있나요?                 없음
4           이 곳의 주소는 어떻게 되나요?  서울 강서구 우장산로16길 26
...                       ...                ...
36055       이 곳의 주소를 알 수 있나요?  서울 동작구 동작대로35길 10
36056           연락처를 알 수 있나요?      070-8876-6424
36057              주차시설이 있나요?                 없음
36058          영업시간은 어떻게 되나요?      12:00 - 22:00
36059  이 곳의 대표적인 메뉴는 무엇이 있나요?        아메리카노, 카페라떼

[36060 rows x 2 columns]


In [5]:
# 기본적인 전처리.
my_data['Source'] = my_data['Source'].str.replace('\W',' ',regex=True)      # 특수 문자 공백으로 대체.
my_data['Source'] = my_data['Source'].str.replace('\s+',' ',regex=True)     # 잉여 공백 제거.
my_data['Source'] = my_data['Source'].apply(lambda x: x.lower()) # 소문자 변환

my_data['Target'] = my_data['Target'].str.replace('\W',' ',regex=True)      # 특수 문자 공백으로 대체.
my_data['Target'] = my_data['Target'].str.replace('\s+',' ',regex=True)     # 잉여 공백 제거.
my_data['Target'] = my_data['Target'].apply(lambda x: x.lower()) # 소문자 변환

In [6]:
# 단어 단위로 토큰화 하고, decoder 입력과 출력에 필요한 Target에는 <sos>태그와 <eos> 태그를 붙여준다.
my_src_in = []
my_tar_in = []
my_tar_out = []
for i in range(my_data.shape[0]):
    a_src = my_data.Source.loc[i].split()           # Encoder에 입력되는 Source는 있는 그대로 split해 둘 수 있다.
    a_tar = my_data.Target.loc[i]
    a_tar_in = [a_word for a_word in ('<sos> ' + a_tar).split()]
    a_tar_out = [a_word for a_word in (a_tar + ' <eos>').split()]
    my_src_in.append(a_src)
    my_tar_in.append(a_tar_in)
    my_tar_out.append(a_tar_out)

In [7]:
# 전처리 이후 출력.
print(my_src_in[10:20])
print(my_tar_in[10:20])
print(my_tar_out[10:20])

[['이', '곳의', '주소를', '알', '수', '있나요'], ['이', '곳의', '대표적인', '메뉴는', '무엇이', '있나요'], ['이', '곳에', '인접한', '시설이', '있나요'], ['연락처가', '어떻게', '되나요'], ['이', '곳의', '대표적인', '메뉴는', '무엇이', '있나요'], ['영업시간을', '알', '수', '있나요'], ['휴무일은', '어떻게', '되나요'], ['영업시간을', '알', '수', '있나요'], ['이', '곳의', '대표적인', '메뉴를', '알', '수', '있나요'], ['연락처를', '알', '수', '있나요']]
[['<sos>', '서울', '강서구', '우장산로16길', '26'], ['<sos>', '생대구탕', '특대구탕'], ['<sos>', '강서구청', '커피바알아', '우리은행'], ['<sos>', '02', '2605', '3248'], ['<sos>', '생대구탕', '특대구탕'], ['<sos>', '10', '00', '22', '00'], ['<sos>', '없음'], ['<sos>', '10', '00', '22', '00'], ['<sos>', '생대구탕', '특대구탕'], ['<sos>', '02', '2605', '3248']]
[['서울', '강서구', '우장산로16길', '26', '<eos>'], ['생대구탕', '특대구탕', '<eos>'], ['강서구청', '커피바알아', '우리은행', '<eos>'], ['02', '2605', '3248', '<eos>'], ['생대구탕', '특대구탕', '<eos>'], ['10', '00', '22', '00', '<eos>'], ['없음', '<eos>'], ['10', '00', '22', '00', '<eos>'], ['생대구탕', '특대구탕', '<eos>'], ['02', '2605', '3248', '<eos>']]


In [8]:
# 정수 인코딩 Source.
my_tokenizer_src = Tokenizer()
my_tokenizer_src.fit_on_texts(my_src_in)
my_encoder_in = my_tokenizer_src.texts_to_sequences(my_src_in)

# 정수 인코딩 Target. In과 Out 별도
my_tokenizer_tar = Tokenizer()
my_tokenizer_tar.fit_on_texts(my_tar_in)         # 학습.
my_tokenizer_tar.fit_on_texts(my_tar_out)        # 추가 학습.
my_decoder_in = my_tokenizer_tar.texts_to_sequences(my_tar_in)
my_decoder_out = my_tokenizer_tar.texts_to_sequences(my_tar_out)

In [9]:
# 패딩.
# 가장 긴 문장에 맞추어진다.
my_encoder_in_pad = pad_sequences(my_encoder_in, padding='post')
my_decoder_in_pad = pad_sequences(my_decoder_in, padding='post')
my_decoder_out_pad = pad_sequences(my_decoder_out, padding='post')

# 다음과 같이 tar_in에만 들어있는 '<sos>'와 '<eos>'가 제대로 학습되었음을 확인해 본다.
print( my_tokenizer_tar.word_index['<sos>'] )
print( my_tokenizer_tar.word_index['<eos>'] )

1
2


In [10]:
# Vocabulary 크기.
SRC_VOCAB_SIZE = len(my_tokenizer_src.word_index) + 1     # 0이 패딩용으로 추가되어 +1.
TAR_VOCAB_SIZE = len(my_tokenizer_tar.word_index) + 1     # 0이 패딩용으로 추가되어 +1.
print('Source vocabulary size : {:d}, Target vocabulary size : {:d}'.format(SRC_VOCAB_SIZE, TAR_VOCAB_SIZE))

Source vocabulary size : 162, Target vocabulary size : 8649


In [11]:
# 변환 사전을 간추려 둔다.
src_to_index = my_tokenizer_src.word_index
index_to_src = my_tokenizer_src.index_word

tar_to_index = my_tokenizer_tar.word_index
index_to_tar = my_tokenizer_tar.index_word

In [12]:
encoder_input_data = np.array(my_encoder_in_pad)
decoder_input_data = np.array(my_decoder_in_pad)
decoder_output_data = np.array(my_decoder_out_pad)

# 인코더 설계
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(SRC_VOCAB_SIZE, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# 디코더 설계
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(TAR_VOCAB_SIZE, 256)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(TAR_VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 모델 컴파일
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# 모델 학습
history = model.fit(
    [encoder_input_data, decoder_input_data], 
    np.expand_dims(decoder_output_data, -1),
    batch_size=64,
    epochs=30,
    validation_split=0.2
)

# 인퍼런스 모델 설계
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

def decode_sequence(input_seq):
    # 입력 문장을 인코딩하여 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <sos> 토큰으로 디코더 입력 시작
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        decoded_sentence += ' ' + sampled_char

        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

# 테스트
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', ' '.join([index_to_src[idx] for idx in input_seq[0] if idx != 0]))
    print('Decoded sentence:', decoded_sentence)


Epoch 1/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 84s 180ms/step - loss: 2.4544 - val_loss: 1.3003
Epoch 2/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 81s 179ms/step - loss: 1.1943 - val_loss: 1.2008
Epoch 3/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - loss: 1.0937 - val_loss: 1.1819
Epoch 4/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - loss: 1.0456 - val_loss: 1.2343
Epoch 5/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - loss: 1.0312 - val_loss: 1.1695
Epoch 6/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - loss: 1.0120 - val_loss: 1.1628
Epoch 7/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 82s 181ms/step - loss: 0.9891 - val_loss: 1.1613
Epoch 8/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 82s 181ms/step - loss: 0.9823 - val_loss: 1.1599
Epoch 9/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 82s 181ms/step - loss: 0.9799 - val_loss: 1.1566
Epoch 10/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 83s 185ms/step - loss: 0.9671 - val_loss: 1.1604
Epoch 11/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 89s 198ms/step - loss: 0.9531 - val_loss: 1.1575
Epoch 12/30
451/451